
---
### 1. Import the libraries that we need


In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from keras.applications import mobilenet
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

### 2. Data preparation

In [2]:
# todo: split manually data into 3 folders. I should automate/script it later...
train_path  = 'data/train'
valid_path  = 'data/valid'
test_path  = 'data/test'

In [3]:
train_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    train_path, target_size=(224,224), batch_size=30)
valid_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    valid_path, target_size=(224,224), batch_size=30)
test_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    test_path, target_size=(224,224), batch_size=30)

Found 400 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


### 3. Download VGGNet and create our new network

In [4]:
base_model = vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (224,224, 3))

In [5]:
# iterate through its layers and lock them to make them not trainable with this code
for layer in base_model.layers:
    layer.trainable = False


In [6]:
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model

# use “get_layer” method to save the last layer of the network
# save the output of the last layer to be the input of the next layer
last_layer = base_model.get_layer('block5_pool')
last_output = last_layer.output

# flatten the classifier input which is output of the last layer of VGG16 model
x = Flatten()(last_output)

# add 2 FC layers, each has 4096 units and relu activation 
x = Dense(64, activation='relu', name='FC_2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
# add our new softmax layer with 3 hidden units
x = Dense(2, activation='softmax', name='softmax')(x)

# instantiate a new_model using keras’s Model class
new_model = Model(inputs=base_model.input, outputs=x)


In [7]:
new_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
new_model.fit_generator(train_batches, steps_per_epoch=4,
                   validation_data=valid_batches, validation_steps=2, epochs=20, verbose=2)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
4/4 - 19s - loss: 1.0799 - accuracy: 0.5417 - val_loss: 1.0066 - val_accuracy: 0.7000
Epoch 2/20
4/4 - 20s - loss: 0.7804 - accuracy: 0.6667 - val_loss: 0.5365 - val_accuracy: 0.8500
Epoch 3/20
4/4 - 20s - loss: 0.8275 - accuracy: 0.6583 - val_loss: 0.7391 - val_accuracy: 0.8000
Epoch 4/20
4/4 - 22s - loss: 0.5879 - accuracy: 0.8000 - val_loss: 0.6675 - val_accuracy: 0.8333
Epoch 5/20
4/4 - 20s - loss: 0.7229 - accuracy: 0.7167 - val_loss: 0.5276 - val_accuracy: 0.8000
Epoch 6/20
4/4 - 20s - loss: 0.4714 - accuracy: 0.7917 - val_loss: 0.8301 - val_accuracy: 0.7500
Epoch 7/20
4/4 - 18s - loss: 0.3742 - accuracy: 0.8300 - val_loss: 0.8601 - val_accuracy: 0.7667
Epoch 8/20
4/4 - 23s - loss: 0.4892 - accuracy: 0.7700 - val_loss: 0.6912 - val_accuracy: 0.8000
Epoch 9/20
4/4 - 21s - loss: 0.3316 - accuracy: 0.8417 - val_loss: 0.3856 - val_accuracy: 0.8500
Epoch 10/20
4/4 - 21s - loss: 0.2780 - accuracy: 0.

In [20]:
def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets

In [24]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np

test_files, test_targets = load_dataset('data/test')

print

In [26]:
test_targets

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.

In [27]:
from keras.preprocessing import image  
from keras.applications.vgg16 import preprocess_input
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_tensors = preprocess_input(paths_to_tensor(test_files))


 55%|████████████████████▉                 | 111/201 [00:00<00:00, 1099.02it/s]

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000020C90252F90>


 55%|████████████████████▉                 | 111/201 [00:12<00:00, 1099.02it/s]

In [23]:
print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*new_model.evaluate(test_tensors, test_targets)))

NameError: name 'test_tensors' is not defined

In [16]:
# evaluate and print test accuracy
score = new_model.evaluate(test_tensors, test_targets)
print('\n', 'Test accuracy:', score[1])

451/451 [==============================] - 40s 89ms/step

 Test accuracy: 0.9556540846824646


### The End